# COVID-19 MIDS Collaboration 

## Data Sourcing: COVID Tracking Project 

This Jupyter Notebook reads in raw data as csv files from a website and exports them as [pickle files for faster loading](https://medium.com/better-programming/load-fast-load-big-with-compressed-pickles-5f311584507e). 

This code was adapted from a script provided to us by Professor Kevin Crook of the Berkeley MIDS program during our W205 (Data Engineering) class. 

### Data sources

US COVID-19 data (historical, at state level) from the COVID tracking project: https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/


### Set up environment 

In [1]:
# Import packages
import pandas as pd
import numpy as np
import io
import requests
import pickle 

### Retrieve data

#### Cases

In [2]:
# get data at URL - this URL is for the state historical data, updated daily at 4pm ET
cases = requests.get("https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_confirmed_usafacts.csv")

In [3]:
# check HTTP request status
cases.status_code

200

In [4]:
# just show the first 2000 characters, the text is really long otherwise
cases.text[0:2000]

'\ufeffcountyFIPS,County Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/2

In [5]:
# load into a Pandas dataframe
cases_covid_df = pd.read_csv(io.StringIO(cases.text))

cases_covid_df

,countyFIPS,County Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20,12/23/20,12/24/20
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,3426,3510,3570,3647,3698,3741,3780,3841,3889,3942
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,11061,11212,11364,11556,11722,11827,11952,12155,12321,12521
3,1005,Barbour County,AL,1,0,0,0,0,0,0,...,1292,1296,1309,1318,1330,1336,1336,1363,1383,1390
4,1007,Bibb County,AL,1,0,0,0,0,0,0,...,1504,1520,1548,1577,1601,1613,1628,1660,1683,1711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3190,56037,Sweetwater County,WY,56,0,0,0,0,0,0,...,2544,2574,2649,2696,2702,2718,2762,2780,2820,2820
3191,56039,Teton County,WY,56,0,0,0,0,0,0,...,1940,1958,1974,1990,1992,2002,2019,2033,2039,2039
3192,56041,Uinta County,WY,56,0,0,0,0,0,0,...,1352,1363,1372,1399,1406,1416,1446,1460,1475,1475
3193,56043,Washakie County,WY,56,0,0,0,0,0,0,...,659,662,665,690,695,699,713,720,726,726


We can see that the format is not ideal - this is a wide dataset with columns tracking cases for every date. Below we convert the dataframe to a long format.

In [6]:
# get names of date columns
date_cols = list(cases_covid_df.columns)[4:len(list(cases_covid_df.columns)) - 1]

# convert wide to long
long_cases_covid_df = pd.melt(cases_covid_df, 
        id_vars = ["countyFIPS", "County Name", "State", "stateFIPS"], value_vars = date_cols, 
        var_name='date', value_name='cases')

# inspect
long_cases_covid_df

,countyFIPS,County Name,State,stateFIPS,date,cases
0,0,Statewide Unallocated,AL,1,1/22/20,0
1,1001,Autauga County,AL,1,1/22/20,0
2,1003,Baldwin County,AL,1,1/22/20,0
3,1005,Barbour County,AL,1,1/22/20,0
4,1007,Bibb County,AL,1,1/22/20,0
...,...,...,...,...,...,...
1076710,56037,Sweetwater County,WY,56,12/23/20,2820
1076711,56039,Teton County,WY,56,12/23/20,2039
1076712,56041,Uinta County,WY,56,12/23/20,1475
1076713,56043,Washakie County,WY,56,12/23/20,726


#### Deaths

In [7]:
# get data at URL - this URL is for the state historical data, updated daily at 4pm ET
deaths = requests.get("https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_deaths_usafacts.csv")

In [8]:
# check HTTP request status 
deaths.status_code

200

In [9]:
# just show the first 2000 characters, the text is really long otherwise
deaths.text[0:2000]

'\ufeffcountyFIPS,County Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/2

In [10]:
# load into a Pandas dataframe
deaths_covid_df = pd.read_csv(io.StringIO(deaths.text))

deaths_covid_df

,countyFIPS,County Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20,12/23/20,12/24/20
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,43,43,43,44,44,44,44,44,46,46
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,141,145,145,146,147,147,147,149,151,151
3,1005,Barbour County,AL,1,0,0,0,0,0,0,...,30,30,30,30,32,32,32,32,32,32
4,1007,Bibb County,AL,1,0,0,0,0,0,0,...,39,42,42,42,42,42,42,42,42,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3190,56037,Sweetwater County,WY,56,0,0,0,0,0,0,...,15,15,15,15,15,15,15,15,15,15
3191,56039,Teton County,WY,56,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2
3192,56041,Uinta County,WY,56,0,0,0,0,0,0,...,7,7,7,7,7,7,7,7,7,7
3193,56043,Washakie County,WY,56,0,0,0,0,0,0,...,11,11,11,11,11,11,11,11,11,11


We can see that the format is not ideal - this is a wide dataset with columns tracking cases for every date. Below we convert the dataframe to a long format.

In [11]:
# get names of date columns
deaths_date_cols = list(deaths_covid_df.columns)[4:len(list(deaths_covid_df.columns)) - 1]

# convert wide to long
long_deaths_covid_df = pd.melt(deaths_covid_df, 
        id_vars = ["countyFIPS", "County Name", "State", "stateFIPS"], value_vars = date_cols, 
        var_name='date', value_name='deaths')

# inspect
long_deaths_covid_df

,countyFIPS,County Name,State,stateFIPS,date,deaths
0,0,Statewide Unallocated,AL,1,1/22/20,0
1,1001,Autauga County,AL,1,1/22/20,0
2,1003,Baldwin County,AL,1,1/22/20,0
3,1005,Barbour County,AL,1,1/22/20,0
4,1007,Bibb County,AL,1,1/22/20,0
...,...,...,...,...,...,...
1076710,56037,Sweetwater County,WY,56,12/23/20,15
1076711,56039,Teton County,WY,56,12/23/20,2
1076712,56041,Uinta County,WY,56,12/23/20,7
1076713,56043,Washakie County,WY,56,12/23/20,11


#### Population

In [12]:
# get data at URL - this URL is for the state historical data, updated daily at 4pm ET
pop = requests.get("https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_county_population_usafacts.csv")


In [13]:
# check HTTP request status 
pop.status_code

200

In [14]:
# just show the first 2000 characters, the text is really long otherwise
pop.text[0:2000]

'\ufeffcountyFIPS,County Name,State,population\r\n0,Statewide Unallocated,AL,0\r\n1001,Autauga County,AL,55869\r\n1003,Baldwin County,AL,223234\r\n1005,Barbour County,AL,24686\r\n1007,Bibb County,AL,22394\r\n1009,Blount County,AL,57826\r\n1011,Bullock County,AL,10101\r\n1013,Butler County,AL,19448\r\n1015,Calhoun County,AL,113605\r\n1017,Chambers County,AL,33254\r\n1019,Cherokee County,AL,26196\r\n1021,Chilton County,AL,44428\r\n1023,Choctaw County,AL,12589\r\n1025,Clarke County,AL,23622\r\n1027,Clay County,AL,13235\r\n1029,Cleburne County,AL,14910\r\n1031,Coffee County,AL,52342\r\n1033,Colbert County,AL,55241\r\n1035,Conecuh County,AL,12067\r\n1037,Coosa County,AL,10663\r\n1039,Covington County,AL,37049\r\n1041,Crenshaw County,AL,13772\r\n1043,Cullman County,AL,83768\r\n1045,Dale County,AL,49172\r\n1047,Dallas County,AL,37196\r\n1049,DeKalb County,AL,71513\r\n1051,Elmore County,AL,81209\r\n1053,Escambia County,AL,36633\r\n1055,Etowah County,AL,102268\r\n1057,Fayette County,AL,16302\r\

In [15]:
# load into a Pandas dataframe
pop_df = pd.read_csv(io.StringIO(pop.text))

# fix column name error
pop_df.columns = ['countyFIPS', 'County Name', 'State', 'population']

pop_df

,countyFIPS,County Name,State,population
0,0,Statewide Unallocated,AL,0
1,1001,Autauga County,AL,55869
2,1003,Baldwin County,AL,223234
3,1005,Barbour County,AL,24686
4,1007,Bibb County,AL,22394
...,...,...,...,...
3190,56037,Sweetwater County,WY,42343
3191,56039,Teton County,WY,23464
3192,56041,Uinta County,WY,20226
3193,56043,Washakie County,WY,7805


### Merge data

In [16]:
# check shape
long_cases_covid_df.shape

(1076715, 6)

In [17]:
# check shape
long_deaths_covid_df.shape

(1076715, 6)

In [ ]:
# merge 3 datasets
covid_df = long_cases_covid_df.merge(long_deaths_covid_df, 
                                         left_on = ["countyFIPS", "County Name", "State", "stateFIPS", "date"],
                                         right_on = ["countyFIPS", "County Name", "State", "stateFIPS", "date"],
                                         how = 'left').merge(pop_df, 
                                                             left_on = ["countyFIPS", "County Name", "State"], 
                                                             right_on = ["countyFIPS", "County Name", "State"], 
                                                             how = 'left').add_prefix('o_') 
# inspect
covid_df.shape

In [ ]:
covid_df.head()

### Convert datatypes

#### Count records

In [ ]:
# count rows and columns
covid_df.shape

#### Check and convert datatypes

We have:

* o_countyFIPS/ stateFIPS: FIPS codes, should be converted from integer to category 
* o_State/ County Name: state & county name; string
* o_date: should be converted from string to date 
* o_cases: integer
* o_deaths/ population: should be converted from float to integer


In [ ]:
# check data types 
covid_df.dtypes

In [ ]:
covid_df.head()

##### String --> Date columns

In [ ]:
# create new version of column as a datetime object - with ymd
covid_df["date"] = pd.to_datetime(covid_df["o_date"])

# check conversion 
covid_df[["o_date", "date"]].head()

##### Integer --> Categorical columns

In [ ]:
# identify which columns to convert 
int_to_category = ["o_countyFIPS", "o_stateFIPS"]
# create new column names
int_to_category_new = list(map(lambda x: x.replace('o_', ''), int_to_category))
# add new converted columns
covid_df[int_to_category_new] = covid_df[int_to_category].apply(lambda x: x.astype('category'))
# check conversion
covid_df[int_to_category + int_to_category_new]


In [ ]:
# check conversion
covid_df[int_to_category + int_to_category_new].dtypes

##### Float --> Integer columns

In [ ]:
# identify which columns to convert 
to_int = list(covid_df.select_dtypes(include = ["float64"]).columns)
# create new column names
converted_to_int = list(map(lambda x: x.replace('o_', ''), to_int))
# add new converted columns
covid_df[converted_to_int] = covid_df[to_int].apply(lambda x: x.astype('Int64'))
# check conversion
covid_df[to_int + converted_to_int].head()


In [ ]:
# check conversion
covid_df[to_int + converted_to_int].dtypes

### Check missingness

We have a lot of missing data in the coumns that track COVID related things. However, we don't have any missing data in the ID columns (date, state, fips). Given the difficulty with retrieving these data, one would expect at least some missing data in these columns. 

These missingness counts are a further validation that the datatype conversions did not introduce additional NAs. 

In [ ]:
covid_df.isnull().sum(axis = 0)

### Generate summary statistics

The summary statistics of original and converted columns are another method for validating the column type conversion. We can see that the conversion did not appear to change summary statistics of the data so from now on we will use the converted columns. 

In [ ]:
# display summary statistics of original columns
covid_df[covid_df.columns[pd.Series(covid_df.columns).str.startswith('o_')]].describe(include = 'all')

In [ ]:
# extract converted columns & unconverted hash column
converted_covid_df = covid_df[list(
    map(lambda x: x.replace('o_', '') if (x != 'o_County Name') & (x != 'o_State') & (x != 'o_cases') else x, 
        list(covid_df[covid_df.columns[pd.Series(covid_df.columns).str.startswith('o_')]].columns)))]
         
# show summary statistics
converted_covid_df.describe(include = 'all')

#### Check duplicates

There are no perfectly duplicated rows. 

In [ ]:
sum(converted_covid_df.duplicated())

There are no duplications for the primary keys - as expected, there appears to be one column for every day, for every state (whether identified via state name or fips code).

In [ ]:
# check duplication w/ date and state name
converted_covid_df[converted_covid_df.duplicated(subset=['date','o_State', 'o_County Name'], keep=False)]

In [ ]:
# check duplication w/ date and state fips code
converted_covid_df[converted_covid_df.duplicated(subset=['date','countyFIPS', 'stateFIPS'], keep=False)]

#### Recount records

In [ ]:
# count rows 
covid_df.shape[0] == converted_covid_df.shape[0]

#### Rename columns

In [ ]:
covid_df.columns = ['countyFIPS', 'county', 'state', 'stateFIPS', 'date', 'cases',
       'deaths', 'population', 'date', 'countyFIPS', 'stateFIPS', 'deaths',
       'population']

covid_df.head()

### Pickle data 

In [ ]:
pickle.dump(converted_covid_df, open( "../Data_pkl/covid19/usafacts_covid_df.pkl", "wb" ) )
